In [8]:
# Import
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

In [9]:
load_dotenv(override=True)

True

In [10]:
# load api_key
openai_api_key = os.getenv('OPENAI_API_KEY')

# load model
MODEL = os.getenv('MODEL')

if openai_api_key:
    print(f"Open API Key exists and begins {openai_api_key[:8]}")

if MODEL:
    print(f"The model {MODEL}")

Open API Key exists and begins sk-proj-
The model gpt-5-nano


In [11]:
# Prompt here

prompt = """
hi there, tell me a joke
"""

In [12]:
# Form the message
messages = [
    {"role": "user", "content": prompt}
]

In [13]:
messages

[{'role': 'user', 'content': '\nhi there, tell me a joke\n'}]

In [14]:
openai = OpenAI()

response = openai.chat.completions.create(
    model = MODEL,
    messages = messages
)

print(response.choices[0].message.content)

Why don’t scientists trust atoms? Because they make up everything. Want another?
